In [85]:
from pyspark.sql import SparkSession

# Create spark session on local machine
spark = SparkSession.builder \
    .master("local") \
    .config("spark.driver.bindAddress","127.0.0.1") \
    .getOrCreate()

In [86]:
from ReviewClass import Review

spark.sparkContext.addPyFile('ReviewClass.py')

22/03/16 19:58:31 WARN SparkContext: The path ReviewClass.py has been added already. Overwriting of added paths is not supported in the current version.


In [87]:
import pandas as pd

# Load data from csv files into dataframe
df1 = pd.read_csv('reviews/train-1.csv')
df2 = pd.read_csv('reviews/train-2.csv')
df3 = pd.read_csv('reviews/train-3.csv')
df4 = pd.read_csv('reviews/train-4.csv')
df5 = pd.read_csv('reviews/train-5.csv')
df6 = pd.read_csv('reviews/train-6.csv')
df7 = pd.read_csv('reviews/train-7.csv')
df8 = pd.read_csv('reviews/train-8.csv')

df_train = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

# Change name of first column
df_train.rename(columns={ df_train.columns[0]: "review_id" }, inplace = True)

In [88]:
# Add data from dataframe into spark session
reviews = [Review(**kwargs) for kwargs in df_train.to_dict(orient='records')]

review_rdd = spark.sparkContext.parallelize(reviews)

In [91]:
# PoC of total character count
total_character_count_per_product_category_id = review_rdd\
    .flatMap(lambda review: [(review.product_category_id, review.ReviewBodyCharCount())])\
    .reduceByKey(lambda count1, count2: count1 + count2)\
    .sortByKey()\
    .collect()

print(total_character_count_per_product_category_id)

[(0, 586944), (1, 223100), (2, 58451), (3, 2086906), (4, 148683), (5, 69346), (6, 1550010), (7, 3770), (8, 5496), (9, 68815), (10, 37976), (11, 104513), (12, 53078), (13, 35703), (14, 19181), (15, 22642), (16, 67598), (17, 23619), (18, 38034), (19, 11432), (20, 13222), (21, 4724), (22, 13235), (23, 2963), (24, 26362), (25, 3123), (26, 4945), (27, 83), (28, 1259)]


22/03/16 19:59:29 WARN TaskSetManager: Stage 12 contains a task of very large size (6671 KiB). The maximum recommended task size is 1000 KiB.
